# DC Controller Backtest

This notebook runs a backtest of the DC (Directional Change) V1 strategy controller.

## 1. Setup and Imports

In [1]:
import warnings
import datetime
import sys
import os
from decimal import Decimal

import pandas as pd
import plotly.express as px

warnings.filterwarnings("ignore")

# Add dc folder to path for imports
dc_path = os.path.dirname(os.path.abspath("03_dc_controller_backtest.ipynb"))
if dc_path not in sys.path:
    sys.path.insert(0, dc_path)

In [2]:
from core.backtesting import BacktestingEngine
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
from dc_controller import DCV1ControllerConfig

## 2. Initialize Backtesting Engine

In [3]:
engine = BacktestingEngine(load_cached_data=True)

## 3. Configure DC Strategy

In [4]:
# Market configuration
connector_name = "binance"
trading_pair = "BTC-USDT"
interval = "1s"


# DC parameters
theta = 0.0075  # 0.75% threshold

# Long-only strategy: buy at UDC, no short positions
long_only = True

# Position sizing
total_amount_quote = 1000

# Risk management - Triple Barrier Method
# Strategy: Buy at UDC, sell when price is 0.4% above UDC with trailing stop
take_profit = 0.004  # 0.4% take profit target
stop_loss = 0.0028     # 0.28% stop loss (wide stop to avoid noise)
time_limit = 60 * 90  # 90 minutes max holding time

# Trailing stop - activates at take_profit level to capture gains
trailing_stop_activation = 0.004  # Activate trailing stop at 0.4% profit
trailing_stop_delta = 0.0005       # Trail by 0.1%

# Execution limits
max_executors_per_side = 1
cooldown_time = 60 * 1  # 1 minutes between trades

# Trading fees (Binance)
trade_cost = 0.001  # 0.1% per side = 0.2% round trip

In [6]:
# Backtest period
start = int(datetime.datetime(2025, 12, 5).timestamp())
end = int(datetime.datetime(2025, 12, 9).timestamp())

print(f"Backtest period: {datetime.datetime.fromtimestamp(start)} to {datetime.datetime.fromtimestamp(end)}")
print(f"Duration: {(end - start) / 86400:.1f} days")

Backtest period: 2025-12-05 00:00:00 to 2025-12-09 00:00:00
Duration: 4.0 days


In [8]:
# Create controller configuration
config = DCV1ControllerConfig(
    connector_name=connector_name,
    trading_pair=trading_pair,
    candles_connector=connector_name,
    candles_trading_pair=trading_pair,
    interval=interval,
    theta=theta,
    long_only=long_only,
    total_amount_quote=Decimal(total_amount_quote),
    take_profit=Decimal(take_profit),
    stop_loss=Decimal(stop_loss),
    trailing_stop=TrailingStop(
        activation_price=Decimal(trailing_stop_activation),
        trailing_delta=Decimal(trailing_stop_delta)
    ),
    time_limit=time_limit,
    max_executors_per_side=max_executors_per_side,
    cooldown_time=cooldown_time,
)

print(f"DC Theta: {config.theta * 100:.2f}%")
print(f"Long-only mode: {config.long_only}")
print(f"Take Profit: {float(config.take_profit) * 100:.2f}%")
print(f"Stop Loss: {float(config.stop_loss) * 100:.2f}%")
print(f"Time Limit: {config.time_limit / 60:.0f} minutes")
print(f"Trade Cost: {trade_cost * 100:.2f}% per side")

DC Theta: 0.75%
Long-only mode: True
Take Profit: 0.40%
Stop Loss: 0.28%
Time Limit: 90 minutes
Trade Cost: 0.10% per side


## 4. Run Backtest

In [9]:
result = await engine.run_backtesting(
    config=config,
    start=start,
    end=end,
    backtesting_resolution=interval,
    trade_cost=trade_cost
)

## 5. Results Summary

In [10]:
print(result.get_results_summary())


Net PNL: $0.59 (0.06%) | Max Drawdown: $-12.35 (-1.23%)
Total Volume ($): 28000.00 | Sharpe Ratio: 0.75 | Profit Factor: 1.02
Total Executors: 14 | Accuracy Long: 0.43 | Accuracy Short: 0.00
Close Types: Take Profit: 6 | Stop Loss: 6 | Time Limit: 2 |
             Trailing Stop: 0 | Early Stop: 0



In [ ]:
result.get_backtesting_figure()

## 6. Executors Analysis

In [ ]:
executors_df = result.executors_df
print(f"Total trades: {len(executors_df)}")
executors_df.head(10)

In [ ]:
# Trade statistics
if len(executors_df) > 0:
    profitable = (executors_df['net_pnl_quote'] > 0).sum()
    total = len(executors_df)
    win_rate = profitable / total * 100
    
    print(f"Win Rate: {win_rate:.1f}% ({profitable}/{total})")
    print(f"Total PnL: {executors_df['net_pnl_quote'].sum():.2f} USDT")
    print(f"Avg PnL per trade: {executors_df['net_pnl_quote'].mean():.2f} USDT")
    print(f"Best trade: {executors_df['net_pnl_quote'].max():.2f} USDT")
    print(f"Worst trade: {executors_df['net_pnl_quote'].min():.2f} USDT")

### PnL per Trade Scatter

In [ ]:
if len(executors_df) > 0:
    executors_df['profitable'] = executors_df['net_pnl_quote'] > 0
    
    fig = px.scatter(
        executors_df,
        x="timestamp",
        y='net_pnl_quote',
        title='PnL per Trade',
        color='profitable',
        color_discrete_map={True: 'green', False: 'red'},
        labels={'timestamp': 'Timestamp', 'net_pnl_quote': 'Net PnL (USDT)'},
        hover_data=['filled_amount_quote', 'side'] if 'side' in executors_df.columns else ['filled_amount_quote']
    )
    
    fig.update_layout(
        xaxis_title="Timestamp",
        yaxis_title="Net PnL (USDT)",
        showlegend=False,
        template="plotly_white"
    )
    
    fig.add_hline(y=0, line_dash="dash", line_color="gray")
    fig.show()

### PnL Distribution

In [ ]:
if len(executors_df) > 0:
    fig = px.histogram(
        executors_df, 
        x='net_pnl_quote', 
        title='PnL Distribution',
        nbins=30
    )
    fig.add_vline(x=0, line_dash="dash", line_color="red")
    fig.update_layout(template="plotly_white")
    fig.show()

### Cumulative PnL

In [ ]:
if len(executors_df) > 0:
    executors_df['cumulative_pnl'] = executors_df['net_pnl_quote'].cumsum()
    
    fig = px.line(
        executors_df,
        x='timestamp',
        y='cumulative_pnl',
        title='Cumulative PnL Over Time'
    )
    fig.add_hline(y=0, line_dash="dash", line_color="gray")
    fig.update_layout(template="plotly_white")
    fig.show()

## 7. Parameter Sensitivity Analysis

Run multiple backtests with different theta values to find optimal parameters.

In [ ]:
# Uncomment to run parameter sweep (takes time)

# thetas_to_test = [0.0025, 0.003, 0.0035, 0.004, 0.0045, 0.005]
# results_by_theta = {}

# for test_theta in thetas_to_test:
#     test_config = DCV1ControllerConfig(
#         connector_name=connector_name,
#         trading_pair=trading_pair,
#         candles_connector=connector_name,
#         candles_trading_pair=trading_pair,
#         interval=interval,
#         theta=test_theta,
#         total_amount_quote=Decimal(total_amount_quote),
#         take_profit=Decimal(take_profit),
#         stop_loss=Decimal(stop_loss),
#         time_limit=time_limit,
#         max_executors_per_side=max_executors_per_side,
#         cooldown_time=cooldown_time,
#     )
#     
#     test_result = await engine.run_backtesting(
#         config=test_config,
#         start=start,
#         end=end,
#         backtesting_resolution=interval,
#         trade_cost=trade_cost
#     )
#     
#     results_by_theta[test_theta] = {
#         'n_trades': len(test_result.executors_df),
#         'total_pnl': test_result.executors_df['net_pnl_quote'].sum() if len(test_result.executors_df) > 0 else 0,
#         'win_rate': (test_result.executors_df['net_pnl_quote'] > 0).mean() * 100 if len(test_result.executors_df) > 0 else 0
#     }
#     print(f"Theta {test_theta*100:.2f}%: {results_by_theta[test_theta]}")

# pd.DataFrame(results_by_theta).T

## Analysis Notes

### Key Questions to Answer:

1. **Signal Quality**
   - How many DC events per day?
   - Win rate for UDC vs DDC signals?

2. **Parameter Sensitivity**
   - How does performance vary with theta?
   - Does higher theta reduce trades but improve quality?

3. **Risk Management**
   - Are stop losses being hit frequently?
   - Is trailing stop capturing gains?
   - Optimal time_limit?

4. **Market Conditions**
   - DC performance in trending vs ranging markets?
   - Different volatility regimes?